In [2]:
import json 
import pandas as pd 
import numpy as np 
import sqlite3 
from datetime import datetime

In [4]:
#Establish connection to the database (database.db file)
con = sqlite3.connect('..\database\sqlite-tools-win32-x86-3380300\database.db')

In [10]:
#Write a query to pull down data, feel free to change this to build your own attributes @Phat
query = '''
    SELECT gen.id as general_tweet_id, klm.id as KLM_reply_tweet_id, 
    gen.timestamp_ms as gen_timestamp_ms, klm.timestamp_ms as klm_timestamp_ms,
    gen.user as gen_user, klm.user as klm_user,
    users.followers_count
    FROM general_tweets as gen, KLM_tweets as klm, users
    
    WHERE gen.id = klm.in_reply_to_status_id AND gen.user_id = users.id AND klm.user_id == '56377143' AND gen.user_id <> '56377143'
'''

In [11]:
#Pull data from database into dataframe
df_data = pd.read_sql_query(query, con)
df_data

,general_tweet_id,KLM_reply_tweet_id,gen_timestamp_ms,klm_timestamp_ms,gen_user,klm_user,followers_count
0,1.1311830432412303e+18,1.1311852592892068e+18,2019-05-22 13:00:28.530,2019-05-22 13:09:16.877,"{'id': 3110268790, 'id_str': '3110268790', 'na...","{'id': 56377143, 'id_str': '56377143', 'name':...",14
1,1.1311830432412303e+18,1.1311852592892068e+18,2019-05-22 13:00:28.530,2019-05-22 13:09:16.877,"{'id': 3110268790, 'id_str': '3110268790', 'na...","{'id': 56377143, 'id_str': '56377143', 'name':...",14
2,1.1311824743804232e+18,1.1311879787710259e+18,2019-05-22 12:58:12.903,2019-05-22 13:20:05.252,"{'id': 1604219274, 'id_str': '1604219274', 'na...","{'id': 56377143, 'id_str': '56377143', 'name':...",17
3,1.1311824743804232e+18,1.1311879787710259e+18,2019-05-22 12:58:12.903,2019-05-22 13:20:05.252,"{'id': 1604219274, 'id_str': '1604219274', 'na...","{'id': 56377143, 'id_str': '56377143', 'name':...",17
4,1.1311830804741243e+18,1.1311880285281075e+18,2019-05-22 13:00:37.407,2019-05-22 13:20:17.115,"{'id': 879493539012296704, 'id_str': '87949353...","{'id': 56377143, 'id_str': '56377143', 'name':...",3
...,...,...,...,...,...,...,...
2397,1.1350952359425393e+18,1.135096594200748e+18,2019-06-02 08:06:07.955,2019-06-02 08:11:31.789,"{'id': 538129774, 'id_str': '538129774', 'name...","{'id': 56377143, 'id_str': '56377143', 'name':...",832
2398,1.1350945810479063e+18,1.1350968290230723e+18,2019-06-02 08:03:31.816,2019-06-02 08:12:27.775,"{'id': 3337513277, 'id_str': '3337513277', 'na...","{'id': 56377143, 'id_str': '56377143', 'name':...",215
2399,1.135095485369389e+18,1.1350974558534124e+18,2019-06-02 08:07:07.423,2019-06-02 08:14:57.223,"{'id': 1918871, 'id_str': '1918871', 'name': '...","{'id': 56377143, 'id_str': '56377143', 'name':...",1662
2400,1.1350975407460639e+18,1.1350996175515075e+18,2019-06-02 08:15:17.463,2019-06-02 08:23:32.612,"{'id': 538129774, 'id_str': '538129774', 'name...","{'id': 56377143, 'id_str': '56377143', 'name':...",832


In [ ]:
#@Marfa: you can probably do the same this as I did there with followers_count to extract other attributes

In [63]:
#Turn back the string of Timestamp data back to Timestamp object. If u don't understand, go ask @Phat
#Feel free to extract other attributes out of this Timestamp object, For example: timezone, day, year,...
df_data['gen_timestamp_ms'] = pd.to_datetime(df_data['gen_timestamp_ms'])
df_data['klm_timestamp_ms'] = pd.to_datetime(df_data['klm_timestamp_ms'])

In [64]:
#Get the reply time/duration between 2 tweets - return Timedelta object
df_data['reply_time'] = df_data['klm_timestamp_ms'] - df_data['gen_timestamp_ms']

In [68]:
#Extract the duration into seconds by attribute .seconds
#Feel free to extract other attibute from this 
df_data['reply_time_in_seconds'] = pd.Series([df_data['reply_time'].iloc[i].seconds for i in range(len(df_data))])

In [49]:
df_data.to_csv('../database/CSV file/conversation.csv')